# Two Cards

Based on Example 2.2.2 in [Introduction to Probability](https://learning.oreilly.com/api/v1/continue/9781466575578/).

A standard deck of cards has 13 Hearts (red), 13 Diamonds (red), 13 Spades (black), and 13 Clubs (black). Assume it is shuffled. Two cards are drawn randomly without replacement. Let A be the event that the first card is a Heart, and B be the event that the second card is red. Find the conditial probabilities of A, given B and of B, given A. I.e., find P(A|B) and P(B|A).

## Model

A deck of card has four suits: Hearts, Diamonds, Spades, and Clubs. Each suit has 13 *ranks*: Ace, 2, 3, ..., 10, Jack, Queen, and King. However, card rank does not occur in the problem description above, so we model a deck just as a set of pairs (s, v) where s is the suit and v is a number in the range 0, 1, .., 12.

In [26]:
from collections import namedtuple
from enum import Enum

Suits = Enum('Suits', 'Diamonds Hearts Clubs Spades')
Card = namedtuple('Card', ['suit', 'value'])

CardDeck = {Card(suit=s, value=n) for s in Suits for n in range(13)}

### Events

The events consist of randomly drawing two cards without replacement. We model events as pairs (c0, c1).  

In [27]:
Event = namedtuple('Event', ['card0', 'card1'])

The function `show` can be used to print `Card` and `Event` objects as human readable strings.

In [28]:
from typing import Union
def show(x: Union[Card, Event]) -> str:
    '''Return a human readable str of x, where x is a Card or Event.
    
    If x is not a Card of Event, str(x) is returned.
    '''
    if type(x) == Card:
        return f"{x.suit.name}-{x.value}"
    elif type(x) == Event:
        return f"({show(x.card0)},{show(x.card1)})"
    return str(x)

### Universe

To compute probabilities, the first step is to specify the *universe* of events. Since we are asked to compute the probabilities of certain events where two cards are drawn without replacement, the universe consists of all possibilities to draw two cards without replacement. We create the universe `E: set` by constructing a set of all possible pairs of cards drawn from `CardDeck` without replacements (i.e., the cards in a pair cannot be the same).

In [29]:
E = {Event(card0=c0, card1=c1) for c0 in CardDeck for c1 in CardDeck if c0 != c1}

In [30]:
# some sanity checks

# The first card is never equal to the second card
assert all(e.card0 != e.card1 for e in E)

# There should be exactly 52 * 51 events in E
assert len(E) == 52 * 51

Recall that A is the event where the first card is a Heart and B is the event where the second card is red. A and B are modelled as sub-sets of E such the elements in A have a Heart as the first card and the elements in B have a red second card.

In [31]:
A_prop = lambda e: e.card0.suit == Suits.Hearts
A = { e for e in E if  A_prop(e)}
B_prop = lambda e: e.card1.suit in {Suits.Hearts, Suits.Diamonds}
B = { e for e in E if B_prop(e) }

The probability of A can be computed by dividing the number of elements in A by the number of elements in E. That is, the probability of A is the ratio |A| / |E|, where |.| gives the number of elements in a set. 

In [32]:
from utils import prob
help(prob)

Help on function prob in module utils:

prob(E: set, S: set) -> fractions.Fraction
    Return the probability of event E in the universe S.
    
    Assumption: E and S are sets and E is a sub-set of S.
    
        E: set The event modelled as a subset of S.
    
        S: set The universe of all events.



In [33]:
print(prob(A, E))
print(prob(B, E))

1/4
1/2


Finally, we can compute the conditional probabilities P(A|B) and P(B|A) requested above.

In [34]:
from utils import cprob
help(cprob)

Help on function cprob in module utils:

cprob(E: set, C: set, S: set) -> fractions.Fraction
    Return the probability of event E conditioned on C in the universe S.
    
    Assumption: E, C, and S are sets and both E and C are sub-sets of S.
    
        E: set The event modelled as a subset of S.
    
        C: set The condition-event modelled as a subset of S.
    
        S: set The universe of all events.



In [35]:
print(cprob(A, B, E))
print(cprob(B, A, E))

25/102
25/51


### Simulation

In order to validate the exact probability computations, it makes sense to develop simulation experiments where cards are actually drawn from the deck in order to see if event A or B or both occurred. If an experiment for event A is done many times, say N>1000, the expectation is that the ratio of "the number of times A happened" over N approximates the probability P(A). The probability of any event can be approximated by simulation, provided we can simulate the event. 

To draw events without replacement, we create a function `pick_events(E,n)`. `E` is a set-like object (Iterable) and `n` is the number of events to pick. The function creates a generator instead of computing the drawn events immediately.

In [11]:
import random

def pick_events(E, n):
    F = [e for e in E]
    while n>0:
        if len(F) == 0:
            raise ValueError("Not enough elements")
        n -= 1
        e = random.choice(F)
        F.remove(e)
        yield e

To use the function `pick_events`, we loop over all elements its generator provides:

In [12]:
def pick1(E):
    '''Helper function to randomly pick one event.'''
    for e in pick_events(E, 1):
        return e

In [13]:
e0 = pick1(E)

In [14]:
print(show(e0))

(Hearts-2,Spades-10)


In [15]:
for i in range(5):
    print("\n".join([show(e) for e in pick_events(E, 1)]))  

(Hearts-10,Clubs-0)
(Hearts-7,Hearts-2)
(Hearts-7,Spades-3)
(Hearts-9,Clubs-7)
(Clubs-5,Hearts-8)


In [16]:
def sample_iter(E, n=1000):
    yield from (pick1(E) for _ in range(n))

In [17]:
from fractions import Fraction

def exp_prob(prop, E, samples):
    return Fraction(len({s for s in samples if prop(s)}), len(samples))

In [53]:
samples = { s for s in sample_iter(E, 1000)}

The `samples` will be used as the universe. With this new universe, we can compute the "experimental probability of A" Aexp and the "experimental probability of B" Bexp. First Aexp and Bexp are defined as the sub-sets of `samples` for which `A_prop` and `B_prop` hold, respectively. Then we use `prop` to compute the probabilities.

In [54]:
Aexp = {e for e in samples if A_prop(e)}
Bexp = {e for e in samples if B_prop(e)}

In [55]:
print(prob(Aexp, samples))
print(prob(Bexp, samples))

187/834
427/834


Similarly, the experimental conditional probabilities P(Aexp|Bexp) and P(Bexp|Aexp) can be computed.

In [56]:
print(cprob(Aexp, Bexp, samples))
print(cprob(Bexp, Aexp, samples))      

92/427
92/187


Let's run a new experiment with more samples.

In [78]:
samples = { s for s in sample_iter(E, 50000)}

In [79]:
Aexp = {e for e in samples if A_prop(e)}
Bexp = {e for e in samples if B_prop(e)}

In [80]:
print(prob(Aexp, samples))
print(prob(Bexp, samples))

1/4
1/2


In [81]:
print(cprob(Aexp, Bexp, samples))
print(cprob(Bexp, Aexp, samples))      

25/102
25/51


Print the probabilities as floats and compare to P(A), P(B), P(A|B), and P(B|A).

In [73]:
print(f"P(A)    = {float(prob(A, E))}")
print(f"P(Aexp) = {float(prob(Aexp, samples))}")

print(f"P(B)    = {float(prob(B, E))}")
print(f"P(Bexp) = {float(prob(Bexp, samples))}")

print(f"P(A|B)       = {float(cprob(A, B, E))}")
print(f"P(Aexp|Bexp) = {float(cprob(Aexp, Bexp, samples))}")

print(f"P(B|A)       = {float(cprob(B, A, E))}")
print(f"P(Bexp|Aexp) = {float(cprob(Bexp, Aexp, samples))}")

P(A)    = 0.25
P(Aexp) = 0.249811320754717
P(B)    = 0.5
P(Bexp) = 0.499622641509434
P(A|B)       = 0.24509803921568626
P(Aexp|Bexp) = 0.24471299093655588
P(B|A)       = 0.49019607843137253
P(Bexp|Aexp) = 0.48942598187311176
